
<a href="https://colab.research.google.com/github/kokchun/Databehandling-21/blob/main/Lectures/L1-missing-aggregation.ipynb" target="_parent"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> &nbsp; for interacting with the code

---
# Lecture notes - Pandas - missing data & aggregations

---
This is the lecture note for **Pandas missing data & aggregations** - but it's built upon contents from previous course: 
- Python programming

<p class = "alert alert-info" role="alert"><b>Note</b> that this lecture note gives a brief introduction to Pandas. I encourage you to read further about pandas.

Read more 

- [documentation - read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

- [documentation - aggregate](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.aggregate.html)

- [documentation - groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)

- [documentation - missing values](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html)
---

## Setup
- create dummy DataFrame
- it's very useful to create dummy DataFrame in order to test and learn Pandas methods

In [114]:
import numpy as np 
import pandas as pd
np.random.seed(42)

n = 8
random_matrix = np.random.randint(1,10, (n,n))
index = np.random.choice(random_matrix.size, 10, replace = False)
print(f"index {index}")
print(f"Flattens the matrix: {random_matrix.ravel()}")

random_matrix = random_matrix*1.0 # to convert it to float as np.nan is float
random_matrix.ravel()[index] = np.nan # returns contiguous flattened array

print(random_matrix)

index [21 20 36  3  5 29 12 37 41 33]
Flattens the matrix: [7 4 8 5 7 3 7 8 5 4 8 8 3 6 5 2 8 6 2 5 1 6 9 1 3 7 4 9 3 5 3 7 5 9 7 2 4
 9 2 9 5 2 4 7 8 3 1 4 2 8 4 2 6 6 4 6 2 2 4 8 7 9 8 5]
[[ 7.  4.  8. nan  7. nan  7.  8.]
 [ 5.  4.  8.  8. nan  6.  5.  2.]
 [ 8.  6.  2.  5. nan nan  9.  1.]
 [ 3.  7.  4.  9.  3. nan  3.  7.]
 [ 5. nan  7.  2. nan nan  2.  9.]
 [ 5. nan  4.  7.  8.  3.  1.  4.]
 [ 2.  8.  4.  2.  6.  6.  4.  6.]
 [ 2.  2.  4.  8.  7.  9.  8.  5.]]


In [115]:
scores = pd.DataFrame(random_matrix, index=[
                      f"Player {i}" for i in range(1, n+1)], columns=[f"Round {i}" for i in range(1,n+1)])
scores

,Round 1,Round 2,Round 3,Round 4,Round 5,Round 6,Round 7,Round 8
Player 1,7.0,4.0,8.0,NaN,7.0,NaN,7.0,8.0
Player 2,5.0,4.0,8.0,8.0,NaN,6.0,5.0,2.0
Player 3,8.0,6.0,2.0,5.0,NaN,NaN,9.0,1.0
Player 4,3.0,7.0,4.0,9.0,3.0,NaN,3.0,7.0
Player 5,5.0,NaN,7.0,2.0,NaN,NaN,2.0,9.0
Player 6,5.0,NaN,4.0,7.0,8.0,3.0,1.0,4.0
Player 7,2.0,8.0,4.0,2.0,6.0,6.0,4.0,6.0
Player 8,2.0,2.0,4.0,8.0,7.0,9.0,8.0,5.0


## Missing data techniques
- None - Pythonic missing data. A Python object - operations done on Python level and not numpy or pandas.
    - Error when performing arithmetic on None dtypes
- NaN - floating point "Not a Number". Supports fast operations, however all arithmetic operations with NaN results in a NaN

Pandas treats NaN and None almost interchangeably
- isnull() - returns true if value is null  
- notnull() - returns true if value is not null
- dropna() - drops the the whole axis with nulls 
- fillna() - fills the null values with certain value

In [116]:
scores.isnull() 

,Round 1,Round 2,Round 3,Round 4,Round 5,Round 6,Round 7,Round 8
Player 1,False,False,False,True,False,True,False,False
Player 2,False,False,False,False,True,False,False,False
Player 3,False,False,False,False,True,True,False,False
Player 4,False,False,False,False,False,True,False,False
Player 5,False,True,False,False,True,True,False,False
Player 6,False,True,False,False,False,False,False,False
Player 7,False,False,False,False,False,False,False,False
Player 8,False,False,False,False,False,False,False,False


In [118]:
scores.dropna() # removes all rows that contains any NaNs

,Round 1,Round 2,Round 3,Round 4,Round 5,Round 6,Round 7,Round 8
Player 7,2.0,8.0,4.0,2.0,6.0,6.0,4.0,6.0
Player 8,2.0,2.0,4.0,8.0,7.0,9.0,8.0,5.0


In [120]:
scores.dropna(axis="columns") # removes all columns that contains any NaNs

,Round 1,Round 3,Round 7,Round 8
Player 1,7.0,8.0,7.0,8.0
Player 2,5.0,8.0,5.0,2.0
Player 3,8.0,2.0,9.0,1.0
Player 4,3.0,4.0,3.0,7.0
Player 5,5.0,7.0,2.0,9.0
Player 6,5.0,4.0,1.0,4.0
Player 7,2.0,4.0,4.0,6.0
Player 8,2.0,4.0,8.0,5.0


In [123]:
scores.fillna(0) # fills all nulls with zero 

,Round 1,Round 2,Round 3,Round 4,Round 5,Round 6,Round 7,Round 8
Player 1,7.0,4.0,8.0,0.0,7.0,0.0,7.0,8.0
Player 2,5.0,4.0,8.0,8.0,0.0,6.0,5.0,2.0
Player 3,8.0,6.0,2.0,5.0,0.0,0.0,9.0,1.0
Player 4,3.0,7.0,4.0,9.0,3.0,0.0,3.0,7.0
Player 5,5.0,0.0,7.0,2.0,0.0,0.0,2.0,9.0
Player 6,5.0,0.0,4.0,7.0,8.0,3.0,1.0,4.0
Player 7,2.0,8.0,4.0,2.0,6.0,6.0,4.0,6.0
Player 8,2.0,2.0,4.0,8.0,7.0,9.0,8.0,5.0


## Missing data - strategy
Note that there are several strategies to deal with missing data and we will come back to this in the future. 
The strategy you should choose may depend on:
- dataset size
    - small dataset - must be more careful with the data
    - large dataset and small amounts of nulls - can probably remove the rows
- valuable information (depends on the usage of the data)
- percentage of missing values 
- domain knowledge - how to fill the nulls 
    - e.g. the score example above might be plausible to assume 0 score if the value is missing for certain round
    - e.g. missing some value in a category might be filled with mean or median of that category
    - e.g. missing value between two points could be interpolated
- note that missing values can negatively impact:
    - data visualization
    - arithmetic computations
    - machine learning algorithms 

---

Kokchun Giang

[LinkedIn][linkedIn_kokchun]

[GitHub portfolio][github_portfolio]

[linkedIn_kokchun]: https://www.linkedin.com/in/kokchungiang/
[github_portfolio]: https://github.com/kokchun/Portfolio-Kokchun-Giang

---